# Nylasum

## Credentials

In [117]:
# Login credentials are also global variables, but these must be set by the user
# Example credentials removed per request of the hackathon coordinators

# Nylas
CLIENT_ID               = ''
CLIENT_SECRET           = ''
ACCESS_TOKEN            = ''
API_URL                 = 'https://api.nylas.com'

# Hugging Face (need access to LLaMa 2 models)
HUGGING_TOKEN           = ''

# Twitter/X
API_KEY                 = ""
API_SECRET              = ""
ACCESS_TOKEN_TWITTER    = ""
ACCESS_SECRET           = ""
BEARER_TOKEN            = r""

## Setup

In [ ]:
!pip install nylas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 5.7 MB/s eta 0:00:00
  Created wheel for nylas: filename=nylas-5.14.1-py3-none-any.whl size=74065 sha256=8520d43f80b33b2efb8ca215a9ac05ae996c938bb6d1f4abfac885e87fc6ccef
  Stored in directory: /root/.cache/pip/wheels/66/71/d4/0c2829e445d7370454218487eb36b64fd58d06c123a50c7321
  Created wheel for urlobject: filename=URLObject-2.4.3-py3-none-any.whl size=14510 sha256=ce5a66c3191a476502ec7538587386ff4e9121b7a7bd468fe12038e86ce800f3
  Stored in directory: /root/.cache/pip/wheels/0d/a2/8a/05c4a3cbe66487af088bc8967fad6de3cc30f4680a5e2e27b8
Successfully built nylas urlobject
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.2
    Uninstalling websocket-client-1.6.2:
      Successfully uninstalled websocket-client-1.6.2


In [ ]:
!pip install tweepy

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00


In [ ]:
# !pip install sentencepiece
!pip install accelerate

In [ ]:
!huggingface-cli login --token $HUGGING_TOKEN

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Database
import sqlite3
from sqlite3 import Error

# Nylas
from nylas import APIClient
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Hugging Face
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Twitter
import tweepy

## Components

### Database

In [ ]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            return conn
            #conn.close()

In [ ]:
connection = create_connection("E:\\sm_app.sqlite")

2.6.0


In [72]:
# SQL QUERY EXECUTION FUNCTIONS

### Executes a query without a return
### NOTE: Only performs a single query
def execute_query(connection, query, query_params):
    cursor = connection.cursor()
    try:
        cursor.execute(query, query_params)
        connection.commit()
        #print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

### Executes a 'read' query (a query with a return)
### NOTE: Only performs a single query
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")


### Should we have a variation of execute query where it performs
### "BEGIN" and "COMMIT" for safety when it modifies the database?

### Executes a query without a return
### NOTE: Only performs a single query
def execute_queries(connection, queries):
    cursor = connection.cursor()
    try:
        cursor.executescript(queries)
        connection.commit()
        #print("Queries executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [ ]:
# GLOBAL TABLE VARIABLES

### usr table
USR_TABLE = "usr"
USR_EMAIL = "email"

### prompt
### May want an index if prompt txt are too long so
### we don't repeat large data in usr_prompt
PROMPT_TABLE = "prompt"
PROMPT_TXT = "txt"
PROMPT_RESPONSE = "response"

### usr_prompt
USR_PROMPT_TABLE = "usr_prompt"
USR_PROMPT_EMAIL = USR_EMAIL
USR_PROMPT_TXT = PROMPT_TXT

In [ ]:
### FUNCTIONS FOR TESTING
def print_responses(responses):
  for response in responses:
      print(response)

def print_table(table):
  query = f"""
    SELECT *
    FROM {table}
  """
  responses = execute_read_query(connection, query)
  print_responses(responses)

In [ ]:
### STRUCTURE QUERY SEARCHING FOR RESPONSE GIVEN USER AND PROMPT

def combine_query_response(usr, txt):
    query = f"""
    SELECT {PROMPT_TABLE}.{PROMPT_RESPONSE}
    FROM {USR_PROMPT_TABLE}, {PROMPT_TABLE}
    WHERE
        {PROMPT_TABLE}.{PROMPT_TXT} = {USR_PROMPT_TABLE}.{USR_PROMPT_TXT}
        AND {USR_PROMPT_TABLE}.{USR_PROMPT_EMAIL} = '{usr}'
        AND {USR_PROMPT_TABLE}.{USR_PROMPT_TXT} = '{txt}';
    """
    return query

In [ ]:
### GET RESPONSE; IF NONE, GENERATE RESPONSE, ADD TO DB, AND RETURN IT
def check_for_value(table: str, search_field: str, value: str, return_field: str):
  query = f"""
    SELECT {table}.{return_field}
    FROM {table}
    WHERE
      {table}.{search_field} = '{value}';
  """
  return execute_read_query(connection, query)

def get_response(usr, txt):
  read_query = combine_query_response(usr, txt)
  response = execute_read_query(connection, read_query)
  if (len(response) > 0):
    print("RESPONSE EXISTS")
    return response

  ### MAY WANT TO ASSUME THAT USERS EXISTS AND
  ### ARE ADDED TO USR TABLE UPON ACCOUNT CREATION
  add_usr_query = f"""
    INSERT OR IGNORE INTO
      {USR_TABLE} ({USR_EMAIL })
    VALUES
      ('{usr}');
  """

  add_usr_prompt_query = f"""
    INSERT OR IGNORE INTO
      {USR_PROMPT_TABLE} ({USR_PROMPT_EMAIL }, {USR_PROMPT_TXT })
    VALUES
      ('{usr}','{txt}');
  """

  # Check if Prompt exists in prompt txt
  response = check_for_value(PROMPT_TABLE, PROMPT_TXT, txt, PROMPT_RESPONSE)
  # If txt exists in prompt, add user-prompt pair and return response
  if (len(response) > 0):
    query = f"""
      BEGIN;
      {add_usr_query}
      {add_usr_prompt_query}
      COMMIT;
    """
    print("PROMPT EXISTS BUT NEW PROMPT FOR USER")
    execute_queries(connection, query)
    return execute_read_query(connection, read_query)

  # Prompt does not exists.
#   generated_response = "place holder" # GENERATE RESPONSE HERE
#   query = f"""
#     BEGIN;

#     {add_usr_query}

#     INSERT INTO
#       {PROMPT_TABLE} ({PROMPT_TXT}, {PROMPT_RESPONSE})
#     VALUES
#       ('{txt}','{generated_response}');

#     {add_usr_prompt_query}

#     COMMIT;
#   """
  return("MESSAGE ID DID NOT EXIST")
#   execute_queries(connection, query)
#   return execute_read_query(connection, read_query)

In [ ]:
### CHECK IF USER EXISTS
# Return -1 if an error occured
# Return 1 if user exists
# Return 0 if user does not exist
# In both cases, print to confirm whether or not user exists (verbose option?)
def check_for_user(email: str):
    check_query = check_for_value(USR_TABLE, USR_EMAIL, email, USR_EMAIL)
    if check_query == None:
      return -1
    usr_exists = len(check_query)
    if usr_exists:
        print(f"USER '{email}' EXISTS")
    else:
        print(f"USER '{email}' DOES NOT EXIST, CONSIDER MAKING AN ACCOUNT")
    return usr_exists

In [ ]:
### CREATE NEW ACCOUNT
# Return -1 if an error occured
# Return 0 if user exists
# Return 1 if user does not exist
# In both cases, print to confirm whether or not user exists
def add_email(connection, email):
    exists = check_for_user(email)
    if exists == None:
      return -1
    if exists:
        return 0
    else:
        print(f'MAKING ACCOUNT FOR {email}')
    insert_email_query = f"""
    BEGIN;
    INSERT INTO
        {USR_TABLE} ({USR_EMAIL})
    VALUES
        ('{email}');
    COMMIT;
    """
    execute_queries(connection, insert_email_query)
    return 1

In [69]:
### CHECK IF MESSAGE ID EXISTS FOR USER
# If yes, return the response for that message ID (prompt)
# If no, return nothing
def check_prompt_response(usr_id, msg_id):
    usr_prompt_query = combine_query_response(usr_id, msg_id)
    if len(execute_read_query(connection, usr_prompt_query)) > 0:
        return get_response(usr_id, msg_id)[0][0]
    return None

In [132]:
create_tables = f"""
  BEGIN;

  CREATE TABLE IF NOT EXISTS {USR_TABLE}
  (
      {USR_EMAIL} VARCHAR(320) PRIMARY KEY
  );

  CREATE TABLE IF NOT EXISTS {PROMPT_TABLE}
  (
      txt TEXT PRIMARY KEY,
      response TEXT
  );

  CREATE TABLE IF NOT EXISTS {USR_PROMPT_TABLE}
  (
      email VARCHAR(320) REFERENCES usr(email),
      txt TEXT REFERENCES prompt(txt),
      PRIMARY KEY (email, txt)
  );

  COMMIT;
"""

execute_queries(connection, create_tables)

### Nylas

In [ ]:
# Use your own credentials
nylas = APIClient(
    CLIENT_ID,
    CLIENT_SECRET,
    ACCESS_TOKEN,
    API_URL
)

In [ ]:
# CONVERT E-MAIL (HTML) TO PLAINfTEXT
def body2txt(body):
    soup = BeautifulSoup(body)
    return soup.get_text('\n')

In [ ]:
def get_subject_from_id(msg_id):
    #print(msg_id)
    message = nylas.messages.get(f"{msg_id}")
    return message.subject

In [ ]:
def get_body_from_id(msg_id):
    #print(msg_id)
    message = nylas.messages.get(f"{msg_id}")
    return body2txt(message.body)

In [ ]:
def get_thread_subjects(list_msgs):
    return [get_subject_from_id(id) for id in list_msgs]

In [63]:
def print_inbox(msg_count):
    messages = nylas.messages.all(limit = msg_count)
    #print(messages)
    for message in messages:
        print("Message ID: {} | Subject: {} | From: {} | Unread: {}".format(
            message.id, message.subject, message.from_, message.unread))

### Large Language Model

In [41]:
# We used Llama-2-7b-chat-hf with max_length of 1024 tokens because it ran
# locally on an NVIDIA T4 GPU (Colab free tier)
# Could use other Meta models with more VRAM:
# - Llama-2-13b-chat-hf
# - Llama-2-70b-chat-hf

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1024,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [42]:
# We used a temperature of 0.0 for consistent answers
# Increasing it will vary responses (try 0.7 for some randomness)
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [115]:
template = """
              Set a length limit of 280 characters for responses.
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain_twittersum = LLMChain(prompt=prompt, llm=llm)

In [110]:
def ask_llm(msg_id):
    msg_body = get_body_from_id(msg_id)
    #print(msg_body)
    print()
    print(get_subject_from_id(msg_id))
    print()

    # Prompt LLM
    response = llm_chain_twittersum.run(msg_body).strip()
    # Clean response for our purposes
    txt_split = response.splitlines()
    for i in range(len(txt_split)):
        txt_split[i] = txt_split[i].strip() + '\n'
    response = ''.join(txt_split)

    print()
    print(response)

    # Add user and prompt, prompt and response to database tables (cached for speed)
    add_query_prompt_response = f"""
    INSERT INTO
        {PROMPT_TABLE} ({PROMPT_TXT}, {PROMPT_RESPONSE})
    VALUES
        (?, ?);
    """
    add_query_prompt_response_params = (msg_id, response)
    execute_query(connection, add_query_prompt_response,
                                add_query_prompt_response_params)
    add_query_usr_prompt = f"""
    INSERT INTO
        {USR_PROMPT_TABLE} ({USR_PROMPT_EMAIL}, {USR_PROMPT_TXT})
    VALUES
        (?, ?);
    """
    add_query_usr_prompt_params = (CLIENT_ID, msg_id)
    execute_query(connection, add_query_usr_prompt,
                                add_query_usr_prompt_params)

    #print_table(USR_PROMPT_TABLE)
    #print_table(PROMPT_TABLE)

### Twitter

In [109]:
# Initialize API
client = tweepy.Client(BEARER_TOKEN, API_KEY, API_SECRET, ACCESS_TOKEN_TWITTER,
                       ACCESS_SECRET)
auth = tweepy.OAuth1UserHandler(API_KEY, API_SECRET, ACCESS_TOKEN_TWITTER,
                                ACCESS_SECRET)
api = tweepy.API(auth)

### Interface

In [121]:
def program_loop():
    while(1):
        print("""What would you like to do?
(1) View Recent Messages
(2) View Message Body
(3) Summarize Message
(4) Tweet Summary
(5) Log Off""")
        print()
        command = int(input("Choice: ").strip())
        print()

        match command:
            case 1:
                # Nylas API defaults to 100 messages
                msg_count = int(input("How many messages? ").strip())
                print()
                print_inbox(msg_count)
                print()

            case 2:
                msg_id = input("Enter Message ID: ").strip()
                print()
                print(get_subject_from_id(msg_id))
                print()
                print(get_body_from_id(msg_id))
                print()

            case 3:
                msg_id = input("Enter Message ID: ").strip()
                response = check_prompt_response(CLIENT_ID, msg_id)
                if response != None:
                    print("RESPONSE CACHED")
                    print(response)
                else:
                    print("RESPONSE NOT CACHED - GENERATING")
                    ask_llm(msg_id)
                print()

            case 4:
                # Tweets will show up on the user's timeline
                # The user must have a Twitter Dev Account
                # For example, here is Zach's account:
                # https://twitter.com/fez_dev_x
                msg_id = input("Enter Message ID: ").strip()
                response = check_prompt_response(CLIENT_ID, msg_id)
                if response != None:
                    print("RESPONSE CACHED")
                    print(response)
                    choice = int(input("Tweet summary? (1 = yes) ").strip())
                    if choice == 1:
                        # Split string every 260 characters - safety against 280-character limit
                        n = 260
                        tweet_list = [response[i:i+n] for i in range(0, len(response), n)]

                        # Release multiple tweets as replies to previous (tweet thread)
                        tweety = client.create_tweet(text = tweet_list[0])
                        num_tweets = 1
                        for i in range(1, len(tweet_list)):
                            tweety = client.create_tweet(text = tweet_list[i],
                                                         in_reply_to_tweet_id = tweety.data['id'])
                            num_tweets += 1

                        print("Sent " + str(num_tweets) + " tweets! Check your account.")
                    else:
                        print("Did not tweet summary.")
                else:
                    print("RESPONSE NOT CACHED")
                print()

            case 5:
                print("Thank you for using Nylasum. Goodbye.")
                # Close Connection to database
                connection.close()
                return

            case _:
                print("Unknown option.")
                print()

## Run Program

In [131]:
connection = create_connection("E:\\sm_app.sqlite")
# If sqlite database is deleted, go to the end of Components > Database to recreate tables
# Link:
# https://colab.research.google.com/drive/1wDfueNLpTSybKaHytgBx9R7A4MYOL6at#scrollTo=hU1mRat6BDI6&line=2&uniqifier=1

2.6.0


In [130]:
added = add_email(connection, CLIENT_ID)

print_table(USR_TABLE)
print()

if added == 1:
    print("E-mail added.")
    print("Welcome to Nylasum " + CLIENT_ID + ".")
elif added == 0:
    print("Welcome back to Nylasum " + CLIENT_ID + ".")

print()
program_loop()

USER '1fm9q5hbu7zsqi3c4numdla9t' DOES NOT EXIST, CONSIDER MAKING AN ACCOUNT
MAKING ACCOUNT FOR 1fm9q5hbu7zsqi3c4numdla9t
('1fm9q5hbu7zsqi3c4numdla9t',)

E-mail added.
Welcome to Nylasum 1fm9q5hbu7zsqi3c4numdla9t.

What would you like to do?
(1) View Recent Messages
(2) View Message Body
(3) Summarize Message
(4) Tweet Summary
(5) Log Off

Choice: 1

How many messages? 5

Message ID: 5j0ud4ya4entfab49qxzjhdv1 | Subject: Meetings for Clubs - Fall 2023 (Resolved | From: [{'email': 'chiefzach2018@gmail.com', 'name': 'Zachary Fernandes'}] | Unread: False
Message ID: itdtidk9p7yxfpq4t1nby9dz | Subject: Meetings for Clubs - Fall 2023 | From: [{'email': 'chiefzach2018@gmail.com', 'name': 'Zachary Fernandes'}] | Unread: False
Message ID: 75fraf2sg2mega6aqcwydo9eh | Subject: New login to X from ChromeDesktop on Windows | From: [{'email': 'verify@x.com', 'name': 'X'}] | Unread: False
Message ID: 91gmp6g6pw9s2ogfpod6is44b | Subject: New login to X from ChromeDesktop on Windows | From: [{'email': '